In [1]:
import json
import requests
import random
import string
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []

        full_dictionary_location = "words_250000_train (1).txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}
        for link in links:
            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word):
        clean_word = word[::2].replace("_", ".")
        len_word = len(clean_word)

        if not self.current_dictionary:
            self.current_dictionary = [dict_word for dict_word in self.full_dictionary if len(dict_word) == len_word]

        filtered_dictionary = [dict_word for dict_word in self.current_dictionary if re.match(clean_word, dict_word)]

        if not filtered_dictionary:
            return self.fallback_guess()

        letter_counts = collections.Counter("".join(filtered_dictionary))
        sorted_letter_count = letter_counts.most_common()
        vowels = "aeiou"

        if len(self.guessed_letters) < 3:
            for letter, _ in sorted_letter_count:
                if letter in vowels and letter not in self.guessed_letters:
                    return letter

        for letter, _ in sorted_letter_count:
            if letter not in self.guessed_letters:
                return letter

        return self.fallback_guess()

    def fallback_guess(self):
        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                return letter

    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            return text_file.read().splitlines()

    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary

        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')

            if verbose:
                print(f"Successfully started a new game! Game ID: {game_id}. # of tries remaining: {tries_remains}. Word: {word}.")

            while tries_remains > 0:
                guess_letter = self.guess(word)
                self.guessed_letters.append(guess_letter)

                if verbose:
                    print(f"Guessing letter: {guess_letter}")

                try:
                    res = self.request("/guess_letter", {"request": "guess_letter", "game_id": game_id, "letter": guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print(f"Server response: {res}")

                status = res.get('status')
                tries_remains = res.get('tries_remains')
                word = res.get('word')

                if status == "success":
                    if verbose:
                        print(f"Game won! Word: {word}")
                    return True
                elif status == "failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print(f"Failed game: {game_id}. Reason: {reason}")
                    return False
                elif status == "ongoing":
                    continue
        else:
            if verbose:
                print("Failed to start a new game")
        return False

    def my_status(self):
        response = self.request("/my_status", {})
        if isinstance(response, list) and len(response) == 4:
            return {
                "total_practice_runs": response[0],
                "total_recorded_runs": response[1],
                "total_recorded_successes": response[2],
                "total_practice_successes": response[3]
            }
        else:
            raise HangmanAPIError(f"Unexpected response format from my_status: {response}")

    def request(self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        if self.access_token:
            args["access_token"] = self.access_token
            if post_args:
                post_args["access_token"] = self.access_token

        time.sleep(0.2)

        for _ in range(5):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if _ + 1 == 5:
                    raise
                time.sleep(1)

        if response.status_code == 200:
            if 'json' in response.headers['content-type']:
                return response.json()
            elif "access_token" in parse_qs(response.text):
                query_str = parse_qs(response.text)
                if "access_token" in query_str:
                    return {"access_token": query_str["access_token"][0]}
            else:
                raise HangmanAPIError('Response content was neither JSON nor query string.')

        if response and isinstance(response, dict) and response.get("error"):
            raise HangmanAPIError(response)
        return response

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        super().__init__(self.result)

# Main execution script
if __name__ == "__main__":
    api = HangmanAPI(access_token="6ab2b69d1083219fdf20cad71fe12e", timeout=2000)

    # Play 41 practice games
    num_games = 41
    wins = 0

    for i in range(num_games):
        print(f"Playing game {i+1}")
        if api.start_game(practice=1, verbose=False):
            wins += 1
        time.sleep(0.5)

    # Calculate and print the success rate for the 41 games
    success_rate = wins / num_games
    print(f'Current session practice success rate: {success_rate:.3f}')

    # Check current stats
    try:
        status = api.my_status()
        total_practice_runs = status["total_practice_runs"]
        total_practice_successes = status["total_practice_successes"]

        # Calculate overall success rate
        overall_success_rate = total_practice_successes / total_practice_runs if total_practice_runs > 0 else 0
        print(f'Overall practice success rate: {overall_success_rate:.3f}')

        # Print detailed stats
        print(f"Total practice runs: {total_practice_runs}")
        print(f"Total practice successes: {total_practice_successes}")
        print(f"Total recorded runs: {status['total_recorded_runs']}")
        print(f"Total recorded successes: {status['total_recorded_successes']}")
    except HangmanAPIError as e:
        print(f"Error retrieving status: {e}")

    # Run additional games if needed to reach 100,000
    remaining_games = 100000 - total_practice_runs
    if remaining_games > 0:
        print(f"Running {remaining_games} more games to reach 100,000 total practice runs")
        for i in range(remaining_games):
            if i % 100 == 0:
                print(f"Playing game {i+1} of {remaining_games}")
            api.start_game(practice=1, verbose=False)
            time.sleep(0.1)

    # Final success rate check
    try:
        final_status = api.my_status()
        final_total_practice_runs = final_status["total_practice_runs"]
        final_total_practice_successes = final_status["total_practice_successes"]
        final_success_rate = final_total_practice_successes / final_total_practice_runs if final_total_practice_runs > 0 else 0
        print(f'Final overall practice success rate: {final_success_rate:.3f}')
    except HangmanAPIError as e:
        print(f"Error retrieving final status: {e}")